In [ ]:
from keras.layers.normalization.batch_normalization import BatchNormalization
!pip install imagecodecs-lite
!pip install imagecodecs
!pip uninstall tiffile
import imagecodecs
from skimage.io import imread, imshow
import numpy as np
import matplotlib.pyplot as plt
import csv
from random import shuffle

from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf


def load_data(path):
    info = []
    images_list = []
    images_class = []
    print('Loading ', path, 'in memory...')
    with open(path, newline='') as csv_file:
        images_reader = csv.reader(csv_file)
        next(images_reader)
        for row in images_reader:
            info.append(row)
    shuffle(info)

    for elem in info:
        images_list.append(imread('/content/drive/MyDrive/Colab Notebooks/Train&Validation/' + elem[0])[55:455, 55:455].reshape(400, 400, 1))
        images_class.append(elem[1] == 'covid')
    return np.array(images_list), np.array(images_class)


def create_model():
    model = Sequential()
    model.add(Conv2D(32, 3, padding="same", activation="relu", input_shape=(400, 400, 1)))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Conv2D(128, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Dropout(0.25))
    

    model.add(Conv2D(256, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())

    model.add(Conv2D(128, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Dropout(0.25))
    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Conv2D(32, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D())

    model.add(Flatten())
    model.add(Dense(2048, activation="relu"))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation="sigmoid"))

    return model


datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 60,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=True)  # randomly flip images


Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
def run_model2(path, epochs_number, folder): #Save acc, loss,... in a file to allow interruption
    try:
        my_model = load_model('/content/drive/MyDrive/Colab Notebooks/' + path)
        print('\033[94m'+'\033[1m' + 'Model Loaded' + '\033[0m')
        my_model.summary()
    except OSError:
        print('\033[91m'+'\033[1m'+'Model', path, 'not found. Creating a new one'+'\033[0m')
        my_model = create_model()
        my_model.summary()
        my_model.compile(loss=tf.keras.losses.BinaryCrossentropy(reduction='sum_over_batch_size'),
                         optimizer=Adam, metrics=["accuracy"])

    train_images, train_classes = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/train' + str(folder) + '.csv')
    validation_images, validation_classes = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/validation' + str(folder) + '.csv')
    datagen.fit(train_images)
    history = my_model.fit(train_images, train_classes, epochs=epochs_number, validation_data=(validation_images,
                                                                                               validation_classes))
    my_model.save('/content/drive/MyDrive/Colab Notebooks/' + path)
    return history.history['accuracy'], history.history['val_accuracy'], history.history['loss'], history.history['val_loss'], epochs_number


def plot_accuracy(accuracy, validation_accuracy, loss, validation_loss, epochs): #Add save as png
    epochs_range = range(epochs)

    plt.figure(figsize=(15, 15))
    plt.subplot(2, 2, 1)
    plt.plot(epochs_range, accuracy, label='Training Accuracy')
    plt.plot(epochs_range, validation_accuracy, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, validation_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

def test_model(model_path, folder):
    my_model = load_model('/content/drive/MyDrive/Colab Notebooks/' + model_path)
    test_images, test_classes = load_data('/content/drive/MyDrive/Colab Notebooks/CSV/test' + str(folder) + '.csv')

    loss, accuracy = my_model.evaluate(test_images, test_classes)
    test = my_model.predict(test_images)
    print("Test accuracy:", str(accuracy * 100) + "%")
    print("Test loss:", str(loss))
    
    TT = 0
    TF = 0
    FT = 0
    FF = 0

    for i in range(0,8322):
      if test[i] == test_classes[i]:
        if test[i] == True:
          TT += 1
        else:
          FF += 1
      else:
        if test[i] == True:
          TF += 1
        else:
          FT += 1
    
    print("TT = " + str(TT) + "TF = " + str(TF) + "FT = " + str(FT) + "FF = " + str(FF))

def predict(model_path, folder):
  my_model = load_model('/content/drive/MyDrive/Colab Notebooks/' + model_path)

#accuracy, validation_accuracy, loss, validation_loss, epochs = run_model2('23-06-22-2.h5', 200, 1)

#plot_accuracy(accuracy, validation_accuracy, loss, validation_loss, epochs)

test_model('23-06-22-1.h5', 1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 31.0 MB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Loading  /content/drive/MyDrive/Colab Notebooks/CSV/test1.csv in memory...
